# 1. Import Libraries

In [19]:
import numpy as np

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt



from sklearn.ensemble import IsolationForest

In [22]:
import eda_helper_functions

# 2. Read the Training Data

In [4]:
file_path = r"C:\Users\Daud Awan\Desktop\Flight-Pridict\Data\train.csv"

train = pd.read_csv(file_path)
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-27,Delhi,Cochin,20:50:00,19:45:00,1375,1.0,No Info,12032
1,Jet Airways,2019-03-27,Delhi,Cochin,09:55:00,04:25:00,1110,1.0,No Info,12242
2,Indigo,2019-05-21,Mumbai,Hyderabad,21:20:00,22:50:00,90,0.0,No Info,2227
3,Vistara,2019-06-01,Banglore,Delhi,21:00:00,23:50:00,170,0.0,No Info,4668
4,Jet Airways,2019-03-06,Delhi,Cochin,17:55:00,04:25:00,2070,2.0,In-flight meal not included,15812
...,...,...,...,...,...,...,...,...,...,...
635,Jet Airways,2019-03-03,Delhi,Cochin,05:25:00,18:50:00,2245,2.0,No Info,20064
636,Multiple Carriers,2019-03-27,Delhi,Cochin,04:00:00,18:50:00,890,1.0,No Info,11299
637,Jet Airways,2019-03-12,Banglore,New Delhi,05:45:00,20:20:00,875,1.0,No Info,13817
638,Jet Airways,2019-03-18,Banglore,New Delhi,14:05:00,05:05:00,900,1.0,No Info,13555


## 2.1 Fixing Data Types

In [5]:
train.dtypes

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration             int64
total_stops        float64
additional_info     object
price                int64
dtype: object

In [6]:
train = train.assign(**{
	col: pd.to_datetime(train.loc[:, col], dayfirst=True, format="mixed")
	for col in ["date_of_journey", "dep_time", "arrival_time"]
})

train.dtypes

airline                    object
date_of_journey    datetime64[ns]
source                     object
destination                object
dep_time           datetime64[ns]
arrival_time       datetime64[ns]
duration                    int64
total_stops               float64
additional_info            object
price                       int64
dtype: object

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          640 non-null    object        
 1   date_of_journey  640 non-null    datetime64[ns]
 2   source           640 non-null    object        
 3   destination      640 non-null    object        
 4   dep_time         640 non-null    datetime64[ns]
 5   arrival_time     640 non-null    datetime64[ns]
 6   duration         640 non-null    int64         
 7   total_stops      640 non-null    float64       
 8   additional_info  640 non-null    object        
 9   price            640 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(4)
memory usage: 50.1+ KB


In [11]:
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-27,Delhi,Cochin,2024-06-25 20:50:00,2024-06-25 19:45:00,1375,1.0,No Info,12032
1,Jet Airways,2019-03-27,Delhi,Cochin,2024-06-25 09:55:00,2024-06-25 04:25:00,1110,1.0,No Info,12242
2,Indigo,2019-05-21,Mumbai,Hyderabad,2024-06-25 21:20:00,2024-06-25 22:50:00,90,0.0,No Info,2227
3,Vistara,2019-06-01,Banglore,Delhi,2024-06-25 21:00:00,2024-06-25 23:50:00,170,0.0,No Info,4668
4,Jet Airways,2019-03-06,Delhi,Cochin,2024-06-25 17:55:00,2024-06-25 04:25:00,2070,2.0,In-flight meal not included,15812
...,...,...,...,...,...,...,...,...,...,...
635,Jet Airways,2019-03-03,Delhi,Cochin,2024-06-25 05:25:00,2024-06-25 18:50:00,2245,2.0,No Info,20064
636,Multiple Carriers,2019-03-27,Delhi,Cochin,2024-06-25 04:00:00,2024-06-25 18:50:00,890,1.0,No Info,11299
637,Jet Airways,2019-03-12,Banglore,New Delhi,2024-06-25 05:45:00,2024-06-25 20:20:00,875,1.0,No Info,13817
638,Jet Airways,2019-03-18,Banglore,New Delhi,2024-06-25 14:05:00,2024-06-25 05:05:00,900,1.0,No Info,13555


## 3. High-level Summary of Data

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          640 non-null    object        
 1   date_of_journey  640 non-null    datetime64[ns]
 2   source           640 non-null    object        
 3   destination      640 non-null    object        
 4   dep_time         640 non-null    datetime64[ns]
 5   arrival_time     640 non-null    datetime64[ns]
 6   duration         640 non-null    int64         
 7   total_stops      640 non-null    float64       
 8   additional_info  640 non-null    object        
 9   price            640 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(4)
memory usage: 50.1+ KB


In [15]:
train.describe(include = "number")

,duration,total_stops,price
count,640.000000,640.000000,640.000000
mean,612.445312,0.810937,9038.831250
std,472.093360,0.652578,4378.178057
min,80.000000,0.000000,1965.000000
25%,175.000000,0.000000,5277.000000
50%,520.000000,1.000000,8454.000000
75%,870.000000,1.000000,12338.500000
max,2280.000000,3.000000,31825.000000


In [16]:
(
	train
	.assign(total_stops=train["total_stops"].astype(object))
	.describe(include="O")
)

,airline,source,destination,total_stops,additional_info
count,640,640,640,640.0,640
unique,8,5,6,4.0,4
top,Jet Airways,Delhi,Cochin,1.0,No Info
freq,235,272,272,354.0,490


# 4. High-level analysis of Missing Values

In [23]:
eda_helper_functions.missing_info(train)

,count,percentage
variable,,


# 5. High-level analysis of Outliers

In [24]:
forest = IsolationForest(n_estimators=10, random_state=42)

In [25]:
(
	train
	.assign(outlier=forest.fit_predict(
		train
		.drop(columns="price")
		.select_dtypes(include="number")
	))
	.query("outlier == -1")
	# .duration
	# .describe()
)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price,outlier
0,Jet Airways,2019-03-27,Delhi,Cochin,2024-06-25 20:50:00,2024-06-25 19:45:00,1375,1.0,No Info,12032,-1
2,Indigo,2019-05-21,Mumbai,Hyderabad,2024-06-25 21:20:00,2024-06-25 22:50:00,90,0.0,No Info,2227,-1
4,Jet Airways,2019-03-06,Delhi,Cochin,2024-06-25 17:55:00,2024-06-25 04:25:00,2070,2.0,In-flight meal not included,15812,-1
7,Jet Airways,2019-06-01,Delhi,Cochin,2024-06-25 14:00:00,2024-06-25 12:35:00,1355,1.0,In-flight meal not included,10262,-1
9,Air Asia,2019-06-06,Kolkata,Banglore,2024-06-25 15:10:00,2024-06-25 23:30:00,500,2.0,No Info,6810,-1
...,...,...,...,...,...,...,...,...,...,...,...
627,Jet Airways,2019-03-27,Mumbai,Hyderabad,2024-06-25 15:50:00,2024-06-25 17:20:00,90,0.0,No Info,5678,-1
633,Jet Airways,2019-06-12,Delhi,Cochin,2024-06-25 11:00:00,2024-06-25 14:15:00,195,0.0,No Info,9564,-1
634,Jet Airways,2019-05-18,Delhi,Cochin,2024-06-25 14:00:00,2024-06-25 12:35:00,1355,1.0,In-flight meal not included,12373,-1
635,Jet Airways,2019-03-03,Delhi,Cochin,2024-06-25 05:25:00,2024-06-25 18:50:00,2245,2.0,No Info,20064,-1
